# OuterSpace

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src.utils import *

In [ ]:
K = 4
M = 5
N = 6

seed = 0

A_KM = Tensor.fromRandom(rank_ids=["K", "M"], shape=[K, M], seed=seed, density=[0.9, 0.5], name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=[0.9, 0.5], name="B")

Note: Small tensors are required for video generation. Partition shapes are decreased for visualization purposes.

In [ ]:
yaml = """
einsum:
    declaration:
        A: [K, M]
        B: [K, N]
        T: [K, M, N]
        Z: [M, N]
    expressions:
        - T[k, m, n] = A[k, m] * B[k, n]
        - Z[m, n] = T[k, m, n]
mapping:
    rank−order:
        A: [K, M]
        B: [K, N]
        T: [M, K, N]
        Z: [M, N]
    partitioning:
        T:
            (K, M): [flatten()]
            KM: [uniform_occupancy(A.4), uniform_occupancy(A.2)]
        Z:
            M: [uniform_occupancy(T.4), uniform_occupancy(T.2)]
    loop−order:
        T: [KM2, KM1, KM0, N]
        Z: [M2, M1, M0, N, K]
    spacetime:
        T:
            space: [KM1, KM0]
            time: [KM2, N]
        Z:
            space: [M1, M0]
            time: [M2, N, K]
"""

compile(yaml)